<a href="https://colab.research.google.com/github/Senbonsakurakageyoshi1/TechMania/blob/main/ControlNet_%E2%80%93_Achieving_Superior_Image_Generation_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!git clone https://github.com/lllyasviel/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 1264, done.
remote: Total 1264 (delta 0), reused 0 (delta 0), pack-reused 1264
Receiving objects: 100% (1264/1264), 122.36 MiB | 19.85 MiB/s, done.
Resolving deltas: 100% (549/549), done.
Updating files: 100% (505/505), done.


In [2]:
%cd ControlNet/
     

/content/ControlNet


In [3]:

%%writefile requirements.txt
xformers
numpy==1.23.1
gradio==3.16.2
albumentations==1.3.0
opencv-contrib-python
imageio==2.9.0
imageio-ffmpeg==0.4.2
pytorch-lightning==1.9.2
omegaconf==2.1.1
test-tube>=0.7.5
streamlit==1.12.1
einops==0.3.0
transformers==4.19.2
webdataset==0.2.5
kornia==0.6
open_clip_torch==2.0.2
invisible-watermark>=0.1.5
streamlit-drawable-canvas==0.8.0
torchmetrics==0.7.0
timm==0.6.12
addict==2.4.0
yapf==0.32.0
prettytable==3.6.0
safetensors==0.2.7
basicsr==1.4.2
einops

Writing requirements.txt


In [4]:
import torch
torch.version.cuda

'11.8'

In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.2/826.2 KB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 K

In [6]:
%cd models
     

/content/ControlNet/models


In [7]:
%%writefile download_models.sh
wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth
#wget https://huggingface.co/lllyasviel/ControlNet/blob/main/models/control_sd15_depth.pth
wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_hed.pth
#wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth
#wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth
#wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth
#wget https://huggingface.co/lllyasviel/ControlNet/blob/main/models/control_sd15_seg.pth

Writing download_models.sh


In [8]:

!sh download_models.sh

--2023-04-05 09:00:19--  https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth
Resolving huggingface.co (huggingface.co)... 52.2.178.255, 34.230.159.254, 52.22.128.237, ...
Connecting to huggingface.co (huggingface.co)|52.2.178.255|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f6/65/f665e553e8fd16131981a96c629e785939f08080bb3cbf8d2b7f5ddbd9e6100a/4de384b16bc2d7a1fb258ca0cbd941d7dd0a721ae996aff89f905299d6923f45?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27control_sd15_canny.pth%3B+filename%3D%22control_sd15_canny.pth%22%3B&Expires=1680944420&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y2LzY1L2Y2NjVlNTUzZThmZDE2MTMxOTgxYTk2YzYyOWU3ODU5MzlmMDgwODBiYjNjYmY4ZDJiN2Y1ZGRiZDllNjEwMGEvNGRlMzg0YjE2YmMyZDdhMWZiMjU4Y2EwY2JkOTQxZDdkZDBhNzIxYWU5OTZhZmY4OWY5MDUyOTlkNjkyM2Y0NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb

In [9]:
%cd ..
     

/content/ControlNet


In [10]:
%%writefile config.py
save_memory = True

Overwriting config.py


In [11]:
with open('gradio_annotator.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_annotator.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_seg2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_seg2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_canny2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_canny2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_depth2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_depth2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_fake_scribble2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_fake_scribble2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_hed2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_hed2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_hough2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_hough2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_normal2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_normal2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_pose2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_pose2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_scribble2image.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_scribble2image.py', 'w') as inputfile:
   inputfile.write(lines)

with open('gradio_scribble2image_interactive.py') as inputfile:
   lines = inputfile.read()
lines = lines.replace("block.launch(server_name='0.0.0.0')", 
                      "block.launch(server_name='0.0.0.0', share=True)")
with open('gradio_scribble2image_interactive.py', 'w') as inputfile:
   inputfile.write(lines)
     


In [12]:
!python gradio_canny2image.py
# !python gradio_seg2image.py
# !python gradio_hed2image.py

logging improved.
Enabled sliced_attention.
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(
ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
S